In [1]:
!git clone https://github.com/nickxir12/MyCLIP_first_repo.git /kaggle/working/open-clip

Cloning into '/kaggle/working/open-clip'...
remote: Enumerating objects: 3678, done.
remote: Counting objects: 100% (3678/3678), done.
remote: Compressing objects: 100% (1411/1411), done.
remote: Total 3678 (delta 2231), reused 3612 (delta 2165), pack-reused 0 (from 0)
Receiving objects: 100% (3678/3678), 15.32 MiB | 36.16 MiB/s, done.
Resolving deltas: 100% (2231/2231), done.


In [76]:
# Step 1: Delete the old repo (if it exists)
!rm -rf /kaggle/working/open-clip  

# Step 2: Clone the latest version from GitHub
!git clone https://github.com/nickxir12/MyCLIP_first_repo.git /kaggle/working/open-clip

Cloning into '/kaggle/working/open-clip'...
remote: Enumerating objects: 3688, done.
remote: Counting objects: 100% (3688/3688), done.
remote: Compressing objects: 100% (1418/1418), done.
remote: Total 3688 (delta 2237), reused 3619 (delta 2168), pack-reused 0 (from 0)
Receiving objects: 100% (3688/3688), 15.33 MiB | 31.51 MiB/s, done.
Resolving deltas: 100% (2237/2237), done.


In [3]:
#!pip install open_clip_torch
!pip install braceexpand
!pip install webdataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 3.1 MB/s eta 0:00:00


In [4]:
!pip install -r /kaggle/working/open-clip/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00


In [77]:
import os
import sys
sys.path.append("/kaggle/working/open-clip/src")
sys.path.append("/kaggle/working/open-clip/src/open_clip")
sys.path.append("/kaggle/working/open-clip/src/open_clip_train/my_metrics")

In [82]:
# import importlib
# import my_metrics
# importlib.reload(my_metrics)
# print(dir(my_metrics))  # Now should include 'batch', 'get_all_embeddings', etc.

In [78]:
from my_metrics import compute_consistency_score,evaluate_model

#Does below work?
from my_metrics import batch,get_all_embeddings

In [7]:
import open_clip
import open_clip_train

In [8]:
import numpy as np
from PIL import Image
from tqdm import tqdm
import json

In [69]:
import torch
import os
import open_clip
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [10]:
#open_clip.list_pretrained()

In [11]:
#                Preparing train datasaet

In [12]:
import json
import pandas as pd

# Load Karpathy JSON file
karpathy_json_path = "/kaggle/input/karpathy-splits/dataset_flickr30k.json"
with open(karpathy_json_path, "r") as f:
    karpathy_data = json.load(f)

# Extract training set
train_data = [item for item in karpathy_data["images"] if item["split"] == "train"]

# Prepare data for CSV
train_records = []
for item in train_data:
    img_filename = f"/kaggle/input/flickr30k/Images/{item['filename']}"
    for sentence in item["sentences"]:
        caption = sentence["raw"]
        train_records.append({"image": img_filename, "caption": caption})

# Convert to DataFrame
df_train = pd.DataFrame(train_records)

# Save to CSV (formatted properly)
csv_path = "/kaggle/working/train_data_karpathy.csv"
df_train.to_csv(csv_path, index=True, index_label="id")

print(f"✅ CSV file saved at: {csv_path}")


✅ CSV file saved at: /kaggle/working/train_data_karpathy.csv


In [13]:
df_train.head()

,image,caption
0,/kaggle/input/flickr30k/Images/1000092795.jpg,Two young guys with shaggy hair look at their ...
1,/kaggle/input/flickr30k/Images/1000092795.jpg,"Two young, White males are outside near many b..."
2,/kaggle/input/flickr30k/Images/1000092795.jpg,Two men in green shirts are standing in a yard.
3,/kaggle/input/flickr30k/Images/1000092795.jpg,A man in a blue shirt standing in a garden.
4,/kaggle/input/flickr30k/Images/1000092795.jpg,Two friends enjoy time spent together.


In [14]:
karpathy_json_path = "/kaggle/input/karpathy-splits/dataset_flickr30k.json"
with open(karpathy_json_path, "r") as f:
    karpathy_data = json.load(f)

# Extract test set
test_data = [item for item in karpathy_data["images"] if item["split"] == "test"]

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [16]:
#Standard loading
# model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
# model = model.to(device)
# model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
# tokenizer = open_clip.get_tokenizer('ViT-B-32')

In [17]:
# FOR CYCLIP PRE TRAINED THAT I HAVE DOWNLOADED LOCALLY
model_clip_3M, _, preprocess = open_clip.create_model_and_transforms(
    model_name="RN50",
    pretrained=None,  # Don't load default weights
    precision='fp32', # or 'amp' for mixed precision
    device=device
)

In [18]:
# Load the checkpoint - OG CLIP - 3M
ckpt = torch.load("/kaggle/input/clip-3m-from-cyclip/clip-3M.pt/best.pt", map_location=device)
state_dict = ckpt["state_dict"]

# Remove 'module.' prefix from keys
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# Load into your model
model_clip_3M.load_state_dict(new_state_dict)
model_clip_3M.to(device)
model_clip_3M.eval()

<ipython-input-18-8c7501dc18c2>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/kaggle/input/clip-3m-from-cyclip/clip-3M.pt/best.pt", map_location=devic

CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (

In [19]:
# FOR CYCLIP PRE TRAINED THAT I HAVE DOWNLOADED LOCALLY
model_Cyclip_3M, _, preprocess = open_clip.create_model_and_transforms(
    model_name="RN50",
    pretrained=None,  # Don't load default weights
    precision='fp32', # or 'amp' for mixed precision
    device=device
)

In [20]:
# Load the checkpoint - CY CLIP - 3M
ckpt = torch.load("/kaggle/input/cyclip-3m/CYCLIP-3M_best.pt", map_location=device)
state_dict = ckpt["state_dict"]

# Remove 'module.' prefix from keys
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# Load into your model
model_Cyclip_3M.load_state_dict(new_state_dict)
model_Cyclip_3M.to(device)
model_Cyclip_3M.eval()

<ipython-input-20-46b685d1e9da>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/kaggle/input/cyclip-3m/CYCLIP-3M_best.pt", map_location=device)


CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (

In [21]:
#Tokenizer for my models
tokenizer = open_clip.get_tokenizer('RN50')

In [22]:
# open_clip.list_pretrained()

In [23]:
#           TO FINE TUNE
#!python /kaggle/working/open-clip/src/open_clip_train/main.py \
#     --train-data /kaggle/working/train_data_karpathy.csv \
#     --dataset-type csv \
#     --csv-img-key image \
#     --csv-caption-key caption \
#     --csv-separator "," \
#     --model ViT-B-32 \
#     --pretrained 'laion2b_s34b_b79k' \
#     --batch-size 48 \
#     --lr 5e-6 \
#     --warmup 1000 \
#     --epochs 1 \
#     --lr-scheduler cosine \
#     --precision amp \
#     --workers 4 \
#     --logs "logs" \
#     --logs "checkpoints" \
#     --save-frequency 1 \
#     --seed 42 \
#     --lambda_dino 0.25 \
#     #--use_soft_labels \
#     --soft_temprature 0.05 \
#     --alpha 0  \

In [24]:
# !ls checkpoints/2025_03_21-13_51_26-model_ViT-B-32-lr_5e-06-b_48-j_4-p_amp/checkpoints  

In [25]:
# import torch
# import open_clip

# # Path to your trained checkpoint
# checkpoint_path = "checkpoints/2025_03_21-13_51_26-model_ViT-B-32-lr_5e-06-b_48-j_4-p_amp/checkpoints/epoch_1.pt"

# # 1. Create the model architecture from scratch (without loading pretrained weights)
# model_ft_mod, preprocess_ft, preprocess_val = open_clip.create_model_and_transforms(
#     "ViT-B-32", pretrained=None
# )

# # 2. Load the trained checkpoint manually
# checkpoint = torch.load(checkpoint_path, map_location="cpu",weights_only=True)

# # If the state_dict is nested under a key (like "state_dict"), adjust accordingly
# if "state_dict" in checkpoint:
#     checkpoint = checkpoint["state_dict"]

# # Load checkpoint weights into model
# model_ft_mod.load_state_dict(checkpoint, strict=False)  # Use strict=False to avoid missing keys error

# # 3. Move model to GPU (if available)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model_ft_mod = model_ft_mod.to(device)
#model_ft_mod.eval()
# print("Model loaded successfully!")

In [26]:
#FROM CYCLIP
def get_all_embeddings(model, all_texts, all_images, root, preprocess, tokenizer, batch_size=1024, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    text_embeddings = []
    image_embeddings = []

    with torch.no_grad():
        dataloader_texts = list(batch(all_texts, batch_size))
        dataloader_images = list(batch(all_images, batch_size))

        bar = zip(dataloader_texts, dataloader_images)
        bar = tqdm(bar, total=len(dataloader_texts), desc="Encoding batches")

        for texts, images in bar:
            # Tokenize text
            text_tokens = tokenizer(texts).to(device)

            # Preprocess and stack images
            image_tensors = torch.stack([
                preprocess(Image.open(os.path.join(root, img)).convert("RGB"))
                for img in images
            ]).to(device)

            # Encode
            image_embedding = model.encode_image(image_tensors)
            text_embedding = model.encode_text(text_tokens)

            # Normalize
            text_embedding /= text_embedding.norm(dim=-1, keepdim=True)
            image_embedding /= image_embedding.norm(dim=-1, keepdim=True)

            text_embeddings.append(text_embedding)
            image_embeddings.append(image_embedding)

        text_embeddings = torch.cat(text_embeddings)
        image_embeddings = torch.cat(image_embeddings)
        return text_embeddings, image_embeddings


In [27]:
#FROM CYCLIP

@torch.no_grad()
def itm_eval(text_embeddings, image_embeddings):

    # sim_matrix_i2t = image_embeddings @ text_embeddings.t()
    # sim_matrix_t2i = text_embeddings @ image_embeddings.t()

    ## Image -> Text
    # ranks = np.zeros(len(sim_matrix_i2t))
    ranks = np.zeros(len(image_embeddings))

    for index in range(0, len(image_embeddings), 5):
        scores = image_embeddings[index] @ text_embeddings.t()
        # scores = sim_matrix_i2t[index]
        li = np.argsort(scores.detach().cpu().numpy())[::-1]
        for i in range(len(li)):
            if index <= li[i] and li[i] <= index + 4:
                rank = i
                break
        ranks[index] = rank
    
        # Compute metrics
    tr1 = 100.0 * len(np.where(ranks < 1)[0]) / len(ranks)
    tr5 = 100.0 * len(np.where(ranks < 5)[0]) / len(ranks)
    tr10 = 100.0 * len(np.where(ranks < 10)[0]) / len(ranks)

    ## Image -> Text
    ranks = np.zeros(len(text_embeddings))
    for index in range(len(text_embeddings)):
        scores = text_embeddings[index] @ image_embeddings.t()
    # for index, scores in tqdm(enumerate(sim_matrix_t2i)):
        scores = scores[::5]
        li = np.argsort(scores.detach().cpu().numpy())[::-1]
        for i in range(len(li)):
            if li[i] == index//5:
                rank = i
                break
        ranks[index] = rank
    
    # Compute metrics
    ir1 = 100.0 * len(np.where(ranks < 1)[0]) / len(ranks)
    ir5 = 100.0 * len(np.where(ranks < 5)[0]) / len(ranks)
    ir10 = 100.0 * len(np.where(ranks < 10)[0]) / len(ranks)
    
    tr_mean = (tr1 + tr5 + tr10) / 3
    ir_mean = (ir1 + ir5 + ir10) / 3
    r_mean = (tr_mean + ir_mean) / 2

    eval_result =  {'txt_r1': tr1,
                'txt_r5': tr5,
                'txt_r10': tr10,
                'txt_r_mean': tr_mean,
                'img_r1': ir1,
                'img_r5': ir5,
                'img_r10': ir10,
                'img_r_mean': ir_mean,
                'r_mean': r_mean}

    return eval_result

In [31]:
def batch(iterable, n = 1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [28]:
test_data = [item for item in karpathy_data["images"] if item["split"] == "test"]
all_captions = []
all_images = []
for item in test_data:
    for sentence in item["sentences"]:
        all_captions.append(sentence["raw"])
        all_images.append(item["filename"])

In [36]:
# Compute embeddings
#FROM CYCLIP

image_root = "/kaggle/input/flickr30k/Images"
text_embeds, image_embeds = get_all_embeddings(
    model_Cyclip_3M, all_captions, all_images, root=image_root,
    preprocess=preprocess, tokenizer=tokenizer, device=device , batch_size=48
)

Encoding batches: 100%|██████████| 105/105 [01:08<00:00,  1.54it/s]


In [37]:
#FROM CYCLIP
# Evaluate
results = itm_eval(text_embeds, image_embeds)
print("📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.2f}")

📊 Evaluation Results:
txt_r1: 87.78
txt_r5: 92.96
txt_r10: 95.10
txt_r_mean: 91.95
img_r1: 26.66
img_r5: 52.84
img_r10: 64.16
img_r_mean: 47.89
r_mean: 69.92


In [81]:
image_root = "/kaggle/input/flickr30k/Images"
text_embeds, image_embeds = get_all_embeddings(
    model_clip_3M, all_captions, all_images, root=image_root,
    preprocess=preprocess, tokenizer=tokenizer, device=device , batch_size=48
)

Encoding batches: 100%|██████████| 105/105 [01:03<00:00,  1.66it/s]


In [83]:
#FROM CYCLIP
# Evaluate
results = itm_eval(text_embeds, image_embeds)
print("📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.2f}")

📊 Evaluation Results:
txt_r1: 88.18
txt_r5: 93.42
txt_r10: 95.56
txt_r_mean: 92.39
img_r1: 26.20
img_r5: 53.32
img_r10: 65.52
img_r_mean: 48.35
r_mean: 70.37


In [38]:
# OG CLIP - 3M
df_results_model = evaluate_model(
    model=model_Cyclip_3M,
    preprocess=preprocess,
    tokenizer=tokenizer,
    test_data=test_data,  # Reuse preloaded test data
    image_folder="/kaggle/input/flickr30k/Images",
    device=device
)

Extracting Features: 100%|██████████| 1000/1000 [00:52<00:00, 18.89it/s]


✅ Extracted 1000 image features and 5000 text features.
Top-1 Accuracy: 26.66%
Top-5 Accuracy: 52.84%
Top-10 Accuracy: 64.16%


In [39]:
# CY CLIP - 3M

df_results_model = evaluate_model(
    model=model_clip_3M,
    preprocess=preprocess,
    tokenizer=tokenizer,
    test_data=test_data,  # Reuse preloaded test data
    image_folder="/kaggle/input/flickr30k/Images",
    device=device
)

Extracting Features: 100%|██████████| 1000/1000 [00:53<00:00, 18.72it/s]


✅ Extracted 1000 image features and 5000 text features.
Top-1 Accuracy: 26.20%
Top-5 Accuracy: 53.32%
Top-10 Accuracy: 65.52%
